In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
feature = np.load('/content/drive/MyDrive/feature.npy')
target = np.load('/content/drive/MyDrive/target.npy')

In [ ]:
feature.shape, target.shape

((269640, 48, 48, 3), (269640, 3))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.applications import VGG16

In [ ]:
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
transfer_model.trainable=False
transfer_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [ ]:
model = Sequential()
model.add(transfer_model)
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 activation (Activation)     (None, 64)                0         
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
 activation_1 (Activation)   (None, 3)                 0

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0002), metrics='accuracy')

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(feature, target,
                                                    test_size=0.2,
                                                    random_state=12,
                                                    stratify=target)

In [ ]:
from kerastuner.tuners import RandomSearch

<ipython-input-12-94471a811b41>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(Flatten(input_shape=(48,48,3)))

    # 하이퍼파라미터로 조정할 레이어 수 및 유닛 수
    for i in range(hp.Int('num_layers', min_value=2, max_value=20)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                               activation='relu'))

    model.add(Dense(3, activation='softmax'))

    # 옵티마이저 및 학습률 설정
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'sgd'])
    hp_learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

    if hp_optimizer == 'adam':
        optimizer = optimizers.Adam(learning_rate=hp_learning_rate)
    else:
        optimizer = optimizers.SGD(learning_rate=hp_learning_rate)

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
# 하이퍼파라미터 튜닝을 위한 튜너 설정 (RandomSearch 사용)
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # 탐색할 하이퍼파라미터 조합 수
    directory='my_tuner_dir',  # 결과를 저장할 디렉토리
    project_name='melon_tuning'  # 프로젝트 이름
)

In [ ]:
# 튜닝 수행
tuner.search(train_X, train_y, epochs=5, batch_size=1024, validation_split=0.2)

# 최적의 하이퍼파라미터 조합 출력
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps}")

# 최적의 모델 가져오기
best_model = tuner.get_best_models(num_models=1)[0]

# 최적의 모델 훈련
best_model.fit(train_X, train_y, epochs=100, batch_size=1024, validation_split=0.2)

Trial 5 Complete [00h 00m 25s]
val_accuracy: 0.45993557572364807

Best val_accuracy So Far: 0.45993557572364807
Total elapsed time: 00h 01m 36s
Best Hyperparameters: <keras_tuner.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7d5e455e82b0>
Epoch 1/100
169/169 [==============================] - 9s 25ms/step - loss: 0.9899 - accuracy: 0.4838 - val_loss: 0.9941 - val_accuracy: 0.4834
Epoch 2/100
169/169 [==============================] - 3s 18ms/step - loss: 0.9702 - accuracy: 0.4990 - val_loss: 0.9630 - val_accuracy: 0.5103
Epoch 3/100
169/169 [==============================] - 2s 14ms/step - loss: 0.9450 - accuracy: 0.5190 - val_loss: 0.9432 - val_accuracy: 0.5224
Epoch 4/100
169/169 [==============================] - 2s 14ms/step - loss: 0.9272 - accuracy: 0.5341 - val_loss: 0.9321 - val_accuracy: 0.5310
Epoch 5/100
169/169 [==============================] - 2s 14ms/step - loss: 0.9014 - accuracy: 0.5519 - val_loss: 0.9126 - val_accuracy: 0.5458
Epoch 6/100
169/169 

In [ ]:
test_loss, test_acc = best_model.evaluate(test_X, test_y)
test_loss, test_acc

1686/1686 [==============================] - 7s 4ms/step - loss: 1.2291 - accuracy: 0.7297


(1.2290809154510498, 0.7296951413154602)

In [ ]:
from keras.models import save_model, load_model

In [ ]:
best_model.save('melon_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# 모델 로드
loaded_model = load_model('melon_model.h5')

In [ ]:
from PIL import Image

In [ ]:
# 이미지 파일 경로
image_path = 'newjeans.jpg'

# 이미지 불러오기
image = Image.open(image_path)

# resize
new_size = (48, 48)
image = image.resize(new_size)

In [ ]:
# 로드된 모델을 사용하여 예측 또는 추가 훈련 등의 작업 수행
prediction = loaded_model.predict(np.expand_dims(image, axis=0))
print(prediction)
np.argmax(prediction)

1/1 [==============================] - 0s 18ms/step
[[0.05103343 0.18057665 0.76838994]]


2

In [ ]:
# 이미지 파일 경로
image_path = 'pussy.jpg'

# 이미지 불러오기
image = Image.open(image_path)

# resize
new_size = (48, 48)
image = image.resize(new_size)

In [ ]:
prediction = loaded_model.predict(np.expand_dims(image, axis=0))
print(prediction)

1/1 [==============================] - 0s 23ms/step
[[0.05681313 0.2808108  0.66237605]]
